## Structured Learning Agent using LangGraph
In this notebook, we will build a structured learning agent using LangGraph. The system will guide learners through a structured learning process of defined but customizable checkpoints. Verifying understanding at each step and providing feynman-style teaching when needed.

## Motivation
- Access to personalized 1:1 tutoring is expensive and not accessible to everyone.
- Provide individualized learning experience to each learner and feedback 24/7.
- Use own notes and web-retrieved content as context.
- Offer patient, simple explanations of complex topics.

## Key Components
1. Learning State Graph: Orchestrates the sequential learning workflow.
2. Checkpoint System: Defines structured learning milestones.
3. Web Search Integration: Dynamically retrives relevant learning materials.
4. Context Processing: Chunks and process learning materials.
5. Question Generation: Creates checkpoint-specific verification questions.
6. Understanding Verification: Evaluates learner comprehension with a clear threshold.
7. Feynman-style Teaching: Provides patient, simple explanations of complex topics.

## Method
The system follows a structured learning cycle.
1. Checkpoint Definition
* Generate sequential learning milestones with clear success criteria.

2. Context Building
* Processes student-provided materials or retrieves relevant web content.
3. Context Validation
* Validates context based on checkpoint criteria.
* Performs additional web searches if context doesn't meet criteria.
4. Embedding Storage
* Stores embeddings for retrieving only relevant chunks during verification.
5. Understanding Verification
* Generates checkpoint-specific questions.
* Evaluates learner's answers against correct answers.
* Provides clear feedback on understanding level.
6. Progressive Learning
* Advances to the next checkpoint when understanding is verified.
* Provides Feynman-style teaching when needed.

### Conclusion
This structured learning agent provides a personalized, 24/7 learning experience. It can be easily extended to include additional features such as progress tracking, personalized recommendations, and more.




## Requirements
#!pip install langchain-community langchain-openai langgraph pydantic python-dotenv semantic-chunkers semantic-router tavily-python

In [7]:
import os 
import operator
import uuid
from typing import Annotated, Dict, List, Optional, Tuple, TypedDict

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from IPython.display import Image, display
from langchain_community.utils.math import cosine_similarity
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from semantic_chunkers import StatisticalChunker
from semantic_router.encoders import OpenAIEncoder

Setup
This agent is implemented using OpenAI's models, but can be used also with self-hosted LLM and embedding models.

In [12]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

tavily_search = TavilySearchResults(max_results=3)
llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

TypeError: str expected, not NoneType